# 06-03

- KNN Clustering 
- Using Categories from KNN to get subset from information
- Continous Features(Only HighLevel Features)
    - MinMax Scaling
    - Remove Constant Features
    - Euclidean Distance
- Categorical Features
    - Hot Encoding
    - Jaccard

In [37]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance, loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf 
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [38]:
transitionMatrices = pd.read_csv('./data/unified/07-transitionsClustered.csv')
transitionMatrices = transitionMatrices[['id','category']]

In [39]:
cover = loadCoverGroups()
listCovers = cover[10]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [40]:
coverSongId

6001248

In [41]:
df = pd.read_csv("./data/unified/05-allDataContinousCategoricalTransition.csv")
df = df.drop(columns=['Unnamed: 0', 'index'])

In [42]:
transitionMatrices

,id,category
0,23,1
1,105,0
2,118,2
3,120,1
4,292,2
...,...,...
5975,6002496,0
5976,6002497,1
5977,6002498,1
5978,6002499,1


In [43]:
df = pd.merge(df,transitionMatrices, on='id')

In [44]:
categoryOriginalSong = df[df.id == originalSongId].category.values[0]
df = df[df.category == categoryOriginalSong]

In [45]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2,category
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
5,407,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
9,469,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
10,588,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
12,635,0.550,0.145,0.0332,0.913000,0.1180,0.138,103.918,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5954,6002477,0.700,0.710,0.0532,0.327000,0.1290,0.582,167.971,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
5964,6002488,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0
5967,6002491,0.665,0.515,0.0442,0.454000,0.3670,0.805,93.082,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
5971,6002495,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0


In [46]:
df = df.drop(columns=['category'])

In [47]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,407,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,469,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10,588,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
12,635,0.550,0.145,0.0332,0.913000,0.1180,0.138,103.918,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5954,6002477,0.700,0.710,0.0532,0.327000,0.1290,0.582,167.971,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5964,6002488,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5967,6002491,0.665,0.515,0.0442,0.454000,0.3670,0.805,93.082,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
5971,6002495,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [48]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)

In [49]:
df

,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,2,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.550,0.145,0.0332,0.913000,0.1180,0.138,103.918,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,0.700,0.710,0.0532,0.327000,0.1290,0.582,167.971,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2166,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2167,0.665,0.515,0.0442,0.454000,0.3670,0.805,93.082,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2168,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [50]:
ids[ids ==originalSongId]

1679    6000352
Name: id, dtype: int64

## Pipeline

In [51]:
dfEuclidean, dfJaccard = getEuclideanJaccardDf(df)

In [52]:
dfEuclidean = dfEuclidean[['acousticness', 'danceability', 'energy', 'liveness', 'speechiness', 'tempo', 'valence']]

In [53]:
dfEuclidean

,acousticness,danceability,energy,liveness,speechiness,tempo,valence
0,0.000458,0.582,0.744,0.0405,0.0336,133.344,0.674
1,0.000013,0.577,0.890,0.0954,0.0345,132.397,0.643
2,0.038000,0.664,0.787,0.1930,0.0239,102.952,0.585
3,0.316000,0.726,0.537,0.0707,0.1140,96.889,0.882
4,0.913000,0.550,0.145,0.1180,0.0332,103.918,0.138
...,...,...,...,...,...,...,...
2165,0.327000,0.700,0.710,0.1290,0.0532,167.971,0.582
2166,0.390000,0.760,0.838,0.1660,0.0510,159.938,0.863
2167,0.454000,0.665,0.515,0.3670,0.0442,93.082,0.805
2168,0.113000,0.555,0.578,0.0747,0.0277,83.382,0.333


In [54]:
# dfEuclidean = applyBoxCoxFeatures(dfEuclidean, ['speechiness', 'acousticness', 'liveness', 'valence'])
dfEuclidean = dfEuclidean.copy()
dfEuclidean = removeConstantFeatures(dfEuclidean)
# dfEuclidean= applyBoxCoxFeatures(dfEuclidean, ['speechiness', 'acousticness', 'liveness', 'valence'])
# dfEuclidean= applyBoxCoxAllFeatures(dfEuclidean)
# dfEuclidean = centerAndScale(dfEuclidean)

In [55]:
scaler = MinMaxScaler()
scaler.fit(dfEuclidean)
dfEuclidean = pd.DataFrame(scaler.transform(dfEuclidean)).copy()

### Original Song & Cover Song

In [56]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfEuclidean)
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfEuclidean)

IndexError: index 0 is out of bounds for axis 0 with size 0

### Get Rankings Euclidean

In [ ]:
ranksDFEuclidean  = applyEuclideanDistance(dfEuclidean, ids, originalSongVector)
ranksDFEuclidean

distance


,id,rank
0,6002043,0.000000
1,6001644,0.170026
2,6002093,0.173253
3,6001831,0.200753
4,6001519,0.206375
...,...,...
2165,6000813,1.427822
2166,6000115,1.458866
2167,6001124,1.465841
2168,1997545,1.469903


In [ ]:
ranksDFEuclidean[ranksDFEuclidean.id == originalSongId]

,id,rank
0,6002043,0.0


In [ ]:
ranksDFEuclidean[ranksDFEuclidean['id']==coverSongId]

,id,rank
219,6002073,0.393433


In [ ]:
ranksDFEuclidean.loc[ranksDFEuclidean.id.isin(listCovers)]

,id,rank
0,6002043,0.000000
219,6002073,0.393433


### Get Rankings Jaccard

In [ ]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfJaccard)
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfJaccard)

In [ ]:
ranksDFJaccard  = applyJaccard(dfJaccard, ids, originalSongVector)

In [ ]:
ranksDFJaccard

,id,rank
0,1983837,0.0
1,6001749,0.0
2,987564,0.0
3,6001878,0.0
4,6000304,0.0
...,...,...
2165,79214,1.0
2166,2530935,1.0
2167,998974,1.0
2168,548740,1.0


In [ ]:
ranksDFJaccard[ranksDFJaccard['id']==coverSongId]

,id,rank
36,6002073,0.333333


In [ ]:
ranksDFJaccard[ranksDFJaccard['id']==coverSongId]

,id,rank
36,6002073,0.333333


### Cover songs

In [ ]:
ranksDFJaccard.loc[ranksDFJaccard.id.isin(listCovers)]

,id,rank
5,6002043,0.000000
36,6002073,0.333333


<font color="red"> TODO: apply weight, or simply add rankings</font>

### Experiment to unify ranks

In [ ]:
ranks = pd.merge(ranksDFJaccard,ranksDFEuclidean, on='id')

In [ ]:
ranks['new_rank'] = ranks['rank_x'] + ranks['rank_y']

In [ ]:
ranks = ranks.sort_values(by='new_rank')
ranks.reset_index(drop=True, inplace=True)

In [ ]:
ranks.head(10)

,id,rank_x,rank_y,new_rank
0,6002043,0.000000,0.000000,0.000000
1,1983837,0.000000,0.248565,0.248565
2,6000304,0.000000,0.248565,0.248565
3,6001878,0.000000,0.324466,0.324466
4,948003,0.000000,0.391372,0.391372
5,6001856,0.000000,0.448176,0.448176
6,6001749,0.000000,0.493164,0.493164
7,987564,0.000000,0.539802,0.539802
8,6001657,0.333333,0.243915,0.577248
9,1137361,0.333333,0.251033,0.584367


In [ ]:
ranks[ranks['id']==coverSongId]

,id,rank_x,rank_y,new_rank
29,6002073,0.333333,0.393433,0.726766


In [ ]:
ranks.loc[ranks.id.isin(listCovers)]

,id,rank_x,rank_y,new_rank
0,6002043,0.000000,0.000000,0.000000
29,6002073,0.333333,0.393433,0.726766
